## Зададим тренировочную и тестовую выборку

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [19]:

classes_train = pd.read_csv('/content/drive/MyDrive/Fedoseev/classes_train.csv', delimiter=',')

In [20]:
classes_train

,is_bot
0,0
1,0
2,0
3,0
4,0
...,...
2395,1
2396,1
2397,1
2398,1


In [21]:
features_train = pd.read_csv('/content/drive/MyDrive/Fedoseev/features_train.csv', delimiter=',')

In [22]:
features_train

,statuses_count,followers_count,friends_count,favourites_count,listed_count,is_default_profile,is_profile_use_background_image,is_verified,user_age,tweets_freq,followers_growth_rate,friends_growth_rate,favourites_growth_rate,listed_growth_rate,followers_friends_ratio,screen_name_length,num_digits_in_screen_name,length_of_name,num_digits_in_name,description_length
0,5124,6641,3074,13768,126,0,0,1,2711,1.890077,2.449650,1.133899,5.078569,0.046477,2.160377,12,0,14,0,73
1,10672,8465,1575,4561,191,0,1,1,4424,2.412297,1.913427,0.356013,1.030967,0.043174,5.374603,11,0,23,0,142
2,704,3108,454,2400,77,0,0,0,1358,0.518409,2.288660,0.334315,1.767305,0.056701,6.845815,12,0,18,0,90
3,6091,304528,526,599,1515,0,1,1,4214,1.445420,72.265781,0.124822,0.142145,0.359516,578.950570,10,0,11,0,51
4,12967,13693,204,1177,452,0,1,1,3200,4.052187,4.279063,0.063750,0.367812,0.141250,67.122549,13,0,14,0,106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,109,13,2,0,5,0,0,0,788,0.138325,0.016497,0.002538,0.000000,0.006345,6.500000,9,0,9,0,124
2396,5503,3,0,0,2,1,1,0,1382,3.981910,0.002171,0.000000,0.000000,0.001447,23524.000000,10,0,18,0,8
2397,2172,35504,0,2,140,1,1,0,1813,1.198014,19.583012,0.000000,0.001103,0.077220,23524.000000,15,0,16,0,130
2398,2513,19,0,0,4,1,1,0,1274,1.972527,0.014914,0.000000,0.000000,0.003140,23524.000000,14,0,17,0,95


In [23]:
X, Y = features_train, classes_train

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, random_state=24)

## 1.1 Обучить модель без каких-либо дополнительных условий, которая должна наилучшим образом отработать на тестовой выборке преподавателя с точки зрения F-меры

In [25]:
import numpy as np
import joblib

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, accuracy_score

In [26]:
# Сохраним показатель точности теста F_measure в виде списка, чтобы 
F_measure = { 
             'DecisionTreeClassifier':0,
             'RandomForestClassifier':0,
             'LogisticRegression':0, 
             'KNeighborsClassifier':0 }
          

In [27]:
DTC = DecisionTreeClassifier()




DTC.fit(X_train, Y_train)
Y_DTC = DTC.predict(X_test)

F_measure['DecisionTreeClassifier'] = f1_score(Y_test, Y_DTC, average = "weighted")

In [28]:
RFC = RandomForestClassifier()




RFC.fit(X_train, Y_train)
Y_RFC = RFC.predict(X_test)

F_measure['RandomForestClassifier'] = f1_score(Y_test, Y_RFC, average = "weighted")

In [29]:
Log = LogisticRegression()
scaler = StandardScaler()

X_train_log = scaler.fit_transform(X_train)
X_test_log = scaler.transform(X_test)


Log.fit(X_train_log, Y_train)
Y_Log = Log.predict(X_test_log)

F_measure['LogisticRegression'] = f1_score(Y_test, Y_Log, average = "weighted")

In [30]:
KNN = KNeighborsClassifier()




KNN.fit(X_train, Y_train)
Y_KNN = KNN.predict(X_test)

F_measure['KNeighborsClassifier'] = f1_score(Y_test, Y_KNN, average = "weighted")

In [31]:
maxim = -1
for i, j in F_measure.items():
    print ('Model:', i, ' F_measure:', j)
    if j > maxim:
        maxim = j
        model_max = i
        
print ('Best model:', model_max, ' F_measure:', maxim)

Model: DecisionTreeClassifier  F_measure: 0.9895390164049233
Model: RandomForestClassifier  F_measure: 0.9916431946739915
Model: LogisticRegression  F_measure: 0.9874647920109871
Model: KNeighborsClassifier  F_measure: 0.9726402229162115
Best model: RandomForestClassifier  F_measure: 0.9916431946739915


In [32]:
# сохраняем модель
joblib.dump(RFC, '/content/drive/MyDrive/Fedoseev/best_model-Ershow.pkl')
best_model = RFC